<a id='top'></a>

#  IBM Q Capture Jupyter Notebook
Author: Christian Lenke, IBM<br>
Version: 2020-11-09

## Table of content

* [Preface](#intro)
* [Introduction](#intro)
<p>
* [Installation, import, function definition](#preparation)
  * [Environment Definition and Customization](#customization)
  * [Get a connection to the Q Capture Server](#connect_qcap)
<p>
* [Q Capture Status overview](#overview_qcap)
  * [General Q Capture overview](#overview_qcap)
  * [Send Queue state](#sendq)
  * [Q Subscriptions by Send Queue](#qsubs_sendq)    
<p>
* [Q Capture performance graphs](#qcap_performance)
  * [Log Reader statistics](#qcap_performance)    
  * [Publisher statistics](#qcap_publ)        
<p>
* [Status details](#qcap_details)
  * [Send Queue details](#qcap_details)
  * [Q Subscription details](#qsubs_details)
  * [Q Subscription quality assurance](#qsubs_qa)        
<p>
* [Q Capture Control Tables](#qcap_cntl)
  * [CAPPARMS](#qcap_cntl)
  * [CAPMON and CAPQMON](#qcap_mon)
  * [CAPTRACE](#qcap_trace)            

[Back to Top](#top)
<a id='intro'></a>

## Preface

This Jupyter Notebook is sample code. No warranty.

## Introduction

This set of Jupyter Notebook can be used to display status, performance, and health of an IBM Q Replication setup. The following components are provided:
 <ul>
  <li>Q Capture Jupyter Notebook (this Notebook)</li>
  <li>Q Apply Jupyter Notebook</li>
  <li>Q Replication function library Jupyter Notebook</li>
  <li>A set of SQL queries in files (asnmonitor) for status calculation and quality assurance</li>    
</ul> 
The Notebooks display the content of the IBM Q Replication control tables. This includes an overall health summary for both Q Capture and Q Apply, performance graphs (throughput, latency, etc.), and control table details for both queues and subscriptions.<br>
Start with the following:<br>
 <ol>
  <li>Copy the set of SQL query files to a location in your environment</li>
  <li>Cuctomize some of the SQL files depending on your environment (more details in the Q Replication function library Notebook)</li>
  <li>Customize the Q Replication function library (details explained in that Notebook)</li>
  <li>Customize the Q Capture and Q Apply Notebook (this Notebook) in section "Environment Definition and Customization" 
</ol> 
To better understand the Q Replication control tables and the SQL queries used, have a look at<br>
<A HREF="https://developer.ibm.com/recipes/tutorials/q-replication-for-dbas/">Q Replication for DBAs</A>,<br> 
<A HREF="https://www.ibm.com/support/knowledgecenter/en/SSTRGZ_11.4.0/com.ibm.swg.im.iis.repl.qrepl.doc/topics/iiyrqctbrcaplist.html">Control tables at the Q Capture server</A>,<br> 
<A HREF="https://www.ibm.com/support/knowledgecenter/en/SSTRGZ_11.4.0/com.ibm.swg.im.iis.repl.qrepl.doc/topics/iiyrqctbrapplist.html">Control tables at the Q Apply server</A>, or <br>
<A HREF="https://www.ibm.com/support/knowledgecenter/en/SSTRGZ_11.4.0/com.ibm.swg.im.iis.db.repl.intro.doc/topics/iiyrqinfroadmap.html">Q Replication Information Roadmap (IBM Knowledge Center)</A>

[Back to Top](#top)
<a id='preparation'></a>

# Preparation

## Installation, import, function definition 

First of all the "IBM Q Replication Monitoring Jupyter Library.ipynb" is run. It imports all required libraries (SQLAlchemy, pandas, pixiedust, ...) and defines functions used in this Notebook.

In [ ]:
%run "IBM Q Replication Monitoring Jupyter Library.ipynb"
# from sqlalchemy import exc

[Back to Top](#top)
<a id='customization'></a>

## Environment Definition and Customization

BEFORE YOU START define your environment here. Specify your individual values for all variables in the below cell.

In [ ]:
# Define your Q Capture Schema. All queries selecting data from the Q Replication control tables 
# have been coded schemaless, the schema needs to be declared once here.
capture_schema = 'ASN'
# capture_schema = 'LSN'

# Database alias of the Q Capture Server (Db2 LUW database name or Db2 z/OS location name)
db2alias = 'SOURCEDB'

# Setting the connection variables for the Q Capture server 
# (usually the Q Replication target database)
db2_driver='ibm_db_sa'
# Host name or ip-address of the Q Capture Server
db2host  = 'localhost'
# Db2 port of the Q Capture Server (defined as char)
db2port  = 50000

# User id privileged to read the Q Capture control tables
db2user  = 'REPLADMIN'
# You will be prompted to type in your password
db2password = getpass.getpass('Password for database ' + db2alias + ': '); 

In [ ]:
# Switch to control debug messages. False: No debug messages; True: Some debug messages will be printed
# debug = True
debug = False

# Display all columns when showing a data frame
pandas.set_option('display.max_columns', None)

# Define the date range for plotting performance data from IBMQREP_CAPMON and IBMQREP_CAPQMON 
# (plots grouped by hour)
# Default: 60 (Retrieve only 60 days of Monitor data for plotting)
# If you want to retrieve all available data, set max_mon_plot_range = -1
# max_mon_plot_range = -1
max_mon_plot_range = 60

# Define the number of most recent messages messages to be retrieved from IBMQREP_CAPTRACE
# If you want to retrieve all available data, set num_messages_trace = -1
# num_messages_trace = -1
num_messages_trace = 200

# Define the date range for fetching monitor information from IBMQREP_CAPMON and IBMQREP_CAPQMON
# Default data rage in number of days, negative number, e.g. -5 (last n days of monitoring data)
default_mon_date_range = 10

# Custom range: -1 means from oldest available (start) to most recent (end).
# Alternatively, specify timestamp in Db2 format
# custom_mon_start_date  = '2018-11-02-10.00.00.000000'
custom_mon_start_date  = -1
custom_mon_end_date    = -1

In [ ]:
# Check the validity of some parms
check_minus_1_or_positive_int('max_mon_plot_range', max_mon_plot_range)        
check_minus_1_or_positive_int('num_messages_trace', num_messages_trace)        
check_positive_int('default_mon_date_range', default_mon_date_range) 

[Back to Top](#top)
<a id='connect_qcap'></a>

## Get a connection to the Q Capture Server

Connect to the Q Capture Server.

In [ ]:
connection = get_connection(db2_driver, db2user, db2password, db2host, str(db2port), db2alias)

Set the CURRENT SCHEMA to the CAPTURE_SCHEMA. All queries executed thereafter are schemaless

In [ ]:
# For the connection the SCHEMA is set to the Q Capture schema
connection.execute("SET CURRENT SCHEMA = '" + capture_schema + "'")

print("INFO: CURRENT SCHEMA = '" + capture_schema + "'")

[Back to Top](#top)
<a id='overview_qcap'></a>

# Q Capture Status overview

## General status overview

The first query displays the overall status of the Q Capture Server. The result set displays messages of the following types:
 <ul>
  <li>C-LAT: Is Q Capture running or not / is the latency as expected (threshold needs to be set in 'qrep_monitor_capture.sql')</li>
    <ul>    
        <li>ERROR: Q Capture not running or latency error threshold exceeded</li>
        <li>WARNING: Q Capture latency warning threshold exceeded</li>
        <li>INFO: Q Capture up and running and latency threshold ok</li>        
    </ul>            
  <li>C-SQU: Send queue status, displayed for each send queue.</li>
    <ul>    
        <li>ERROR: Send Queue inactive due to an error</li>
        <li>INFO: Send Queue active</li>            
    </ul>                    
  <li>S-SUB: Summary whether all subscriptions for a queue are active or not, displayed for each send queue.</li>
</ul> 

In [ ]:
# The function get_capture_status executes the query 'qrep_monitor_capture.sql' 
# (with language dependent result set) and returns a pandas data frame
df_capstate = get_capture_status(connection)

# drop column 'CURRENT_SERVER'
df_capstate.drop(df_capstate.columns[2], axis=1, inplace=True)

# conditional formatting for column 'sev'
df_capstate = (df_capstate.style
    .applymap(sev_background, subset=['sev'])
    .applymap(sev_foreground, subset=['sev'])
)

# printing the data frame
df_capstate

[Back to Top](#top)
<a id='sendq'></a>

### Overall send queue state

Number of active send queues (green) vs. number of inactive send queues (red)

In [ ]:
# The function get_sendq_state executes a query which counts active vs. inactive queues
# and returns a pandas data frame
df_qstate = get_sendq_state(connection)  

df_qstate.set_index('queues',inplace=True)

qpl = df_qstate.plot.barh(stacked=True,color=['green', 'red'], 
                          title="Active vs. inactive send queues",
                          figsize=(10,2))

qpl.set_xlabel("Number of queues")
myvar = qpl.set_ylabel("")

[Back to Top](#top)
<a id='qsubs_sendq'></a>

## Subscription STATE by send queue

Number of active subscriptions (green) vs. number of inactive subscriptions (red) per send queue

In [ ]:
# The function get_substate_by_recvq executes a query which counts active vs. inactive queues
# and returns a pandas data frame
df_q = get_substate_by_sendq(connection)

numqs = len(df_q.sendq.unique())
if numqs >=20:
    calc_fig_height = numqs / 2
if numqs <=2:
    calc_fig_height = numqs * 2
else:
    calc_fig_height =  numqs

# NEXT: Adding labels (SENDQ) - does not work
df_q.set_index('sendq',inplace=True) 

sqpl = df_q.plot.barh(stacked=True,color=['green', 'red'],
                       title="Number or active vs. inactive subs per send queue",
                       figsize=(15,calc_fig_height))

sqpl.set_xlabel("Number of subs")
myvar = sqpl.set_ylabel("")

[Back to Top](#top)
<a id='qcap_performance'></a>

# Q Capture Performance Graphs

## Capture Log Reader statistics per hour (rows_processed, current_memory, trans_spilled)

The following codes creates plots which display Q Replication log reader KPIs grouped by hour. First, all data available in IBMQREP_CAPMON is evaluated (can be limited with parameter <i>max_mon_plot_range</i>). Next, the most recent 20 days will be evaluated, lastly the most recent 5 days.<p>
The plots show the maximum memory used in the particular hour, the summary of all rows  processed from the log, and the number of monster transactions (which exceeded Q Capture's MEMORY_LIMNIT).

In [ ]:
# The function get_perf_logrd executes a query which selects data from IBMQREP_CAPMON
# and returns a pandas data frame. The CAPMON data is GROUPed BY hour(monitor_time)
# by the query. In case max_mon_plot_range == 0, the query will retrieve all available data 
# from CAPMON. If max_mon_plot_range > 0, the CAPMON data will be limited to the recent
# max_mon_plot_range days

if max_mon_plot_range == -1:              
    print('INFO: Evaluating all available CAPMON data') 
else:   
    print('INFO: Limiting the CAPMON data to a maximum of ' + str(max_mon_plot_range) + ' days.')
    
capmon_logrd = get_perf_logrd(connection, max_mon_plot_range)

In [ ]:
# DEBUG - print sample data of the data frame retrieved from APPLYMON
if debug:
   print(capmon_logrd)

In [ ]:
# Todo: What if data fame is empty (test)

# Determine the date of max(monitor_time). This is used when only a subset (e.g., most recent
# 5 days of data) is displayed later. Could be that the Notebook is used to evaluate saved 
# CAPMON data, so that today() - n days would have no data. 
maxmontime = capmon_logrd['monitor_date'].max()
maxmondate = datetime.strptime(maxmontime[:10], "%Y-%m-%d")

minmontime = capmon_logrd['monitor_date'].min()
minmondate = datetime.strptime(minmontime[:10], "%Y-%m-%d")

print('INFO: CAPMON' + ' [' + str(minmondate)[:10] + ' - ' + str(maxmondate)[:10] + ']') 

All available data from IBMQREP_CAPMON (max. <i>max_mon_plot_range</i> days)

In [ ]:
# Plot figure with all available data   

# deltadays = 0 means all available data of the data frame
deltadays = max_mon_plot_range

if deltadays == -1:
    print('INFO: Plotting range: All available data from ' + minmondate.strftime("%Y-%m-%d") 
                                                           + ' up to ' + maxmondate.strftime("%Y-%m-%d"))
else:
    print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                           + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                           + maxmondate.strftime("%Y-%m-%d"))
    
# The function plot_capmon creates a figure with 1 plot from the 
# data frame capmon_lat
capmon_plt1 = plot_capmon(capmon_logrd,deltadays)
capmon_plt1.show()  

Recent 20 days from IBMQREP_CAPMON

In [ ]:
# Zoom in - Same figure as above, with data of the last 20 days only (maxmondate - 20 days)

# display 20 days of data only
deltadays = 20
# old: date_20_days_ago = date.today() - timedelta(days=deltadays)
date_20_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
   print('DEBUG: date_20_days_ago.strftime("%m/%d/%Y"): ' + date_20_days_ago.strftime("%Y-%m-%d"))

print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                       + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                       + maxmondate.strftime("%Y-%m-%d"))

# New data frame - limited by the calculated data range 
#   - reset_index was introduced to guarantee xticks in the plot
capmon_logrd_20 = capmon_logrd[capmon_logrd.monitor_date 
                               >= date_20_days_ago.strftime("%Y-%m-%d")].reset_index(drop=True)  

capmon_plt2 = plot_capmon(capmon_logrd_20,deltadays)
capmon_plt2.show()        

Recent 5 days from IBMQREP_CAPMON

In [ ]:
# Zoom in - Same figure as above, with data of the last 5 days only (maxmondate - 5 days)

# display 5 days of data only
deltadays = 5

# old: date_5_days_ago = date.today() - timedelta(days=deltadays)
date_5_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
   print('DEBUG: date_5_days_ago.strftime("%m/%d/%Y"): ' + date_20_days_ago.strftime("%Y-%m-%d"))

print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                       + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                       + maxmondate.strftime("%Y-%m-%d"))

# New data frame - limited by the calculated data range 
#   - reset_index was introduced to guarantee xticks in the plot
capmon_logrd_5 = capmon_logrd[capmon_logrd.monitor_date 
                              >= date_5_days_ago.strftime("%Y-%m-%d")].reset_index(drop=True)  

capmon_plt3 = plot_capmon(capmon_logrd_5,deltadays)
capmon_plt3.show()   

[Back to Top](#top)
<a id='qcap_publ'></a>

## Capture Publisher statistics per hour (rows_processed, mq_messages ,xmitqdepth)

The following codes creates plots which display Q Replication publisher KPIs grouped by hour. First, all data available in IBMQREP_CAPQMON is evaluated (can be limited with parameter max_mon_plot_range). Next, the most recent 20 days will be evaluated, lastly the most recent 5 days.<p>
The plots show the maximum XMIT queue depth in the particular hour, and the summary of all rows published to MQ and the summary of all MQ messages sent by hour.

In [ ]:
# The function get_perf_logrd executes a query which selects data from IBMQREP_CAPMON
# and returns a pandas data frame. The CAPMON data is GROUPed BY hour(monitor_time)
# by the query. In case max_mon_plot_range == 0, the query will retrieve all available data 
# from CAPQMON. If max_mon_plot_range > 0, the CAPQMON data will be limited to the recent
# max_mon_plot_range days

if max_mon_plot_range == -1:              
    print('INFO: Evaluating all available CAPQMON data') 
else:   
    print('INFO: Limiting the CAPQMON data to ' + str(max_mon_plot_range) + ' days.')

capqmon_publ = get_perf_publ(connection,max_mon_plot_range)

In [ ]:
# DEBUG - print sample data of the data frame retrieved from APPLYMON
if debug:
   print(capqmon_publ)

In [ ]:
# Todo: What if data fame is empty (test)

# Determine the date of max(monitor_time). This is used when only a subset (e.g., most recent
# 5 days of data) is displayed later. Could be that the Notebook is used to evaluate saved 
# APPLYMON data, so that today() - n days would have no data. 
maxmontime = capqmon_publ['monitor_date'].max()
maxmondate = datetime.strptime(maxmontime[:10], "%Y-%m-%d")

maxmontime = capqmon_publ['monitor_date'].min()
maxmondate = datetime.strptime(maxmontime[:10], "%Y-%m-%d")

print('INFO: CAPMON' + ' [' + str(minmondate)[:10] + ' - ' + str(maxmondate)[:10] + ']') 
print()

# Determination of the number of distinct receive queues in the result set
numqs = len(capqmon_publ.sendq.unique())

print('INFO: numqs=' + str(numqs))

# print the distinct names and data ranges of the queues
for i in range(0,numqs):
    minmontimeq = capqmon_publ[capqmon_publ.sendq == capqmon_publ.sendq.unique()[i]]['monitor_date'].min()
    maxmontimeq = capqmon_publ[capqmon_publ.sendq == capqmon_publ.sendq.unique()[i]]['monitor_date'].max()
    minmondateq = datetime.strptime(minmontimeq[:10], "%Y-%m-%d")                
    maxmondateq = datetime.strptime(maxmontimeq[:10], "%Y-%m-%d")        
    print('INFO: ' + capqmon_publ.sendq.unique()[i] + ' [' + str(minmondateq)[:10] + ' - ' 
                   + str(maxmondateq)[:10] + ']')

All available data from IBMQREP_CAPQMON (max. <i>max_mon_plot_range</i> days)

In [ ]:
# Plot figure (one plot per queue) with all available data   

# deltadays = 0 means all available data of the data frame
deltadays = max_mon_plot_range
if deltadays == -1:
    date_max_days_ago = minmondate
else:
    date_max_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
    print('DEBUG: date_max_days_ago.strftime("%Y-%m-%d"): ' + date_max_days_ago.strftime("%Y-%m-%d"))        
    
if deltadays == -1:
    print('INFO: Plotting range: All available data from ' + minmondate.strftime("%Y-%m-%d") 
                                                           + ' up to ' + maxmondate.strftime("%Y-%m-%d"))
else:
    print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                           + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                           + maxmondate.strftime("%Y-%m-%d"))    
    
# The function plot_capmon creates a figure with n plots (n = number of queues) from the 
# data frame capqmon_lat
# The hight of the following figures (calculated in plot_capqmon) depends on the number of distinct 
# receive queues. The more queues, the less space per individual queue (to limit the size of the figure)
capqmon_plt1 = plot_capqmon(capqmon_publ,numqs,deltadays)
capqmon_plt1.show() 

Recent 20 days from IBMQREP_CAPQMON

In [ ]:
# Zoom in - Same figure as above, with data of the last 20 days only (maxmondate - 20 days)

# display 20 days of data only
deltadays = 20
# old: date_20_days_ago = date.today() - timedelta(days=deltadays)
date_20_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
   print('DEBUG: date_20_days_ago.strftime("%m/%d/%Y"): ' + date_20_days_ago.strftime("%Y-%m-%d"))

print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                       + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                       + maxmondate.strftime("%Y-%m-%d"))    
    
# New data frame - limited by the calculated data range
#   - reset_index was introduced to guarantee xticks in the plot
capqmon_publ_20 = capqmon_publ[capqmon_publ.monitor_date 
                               >= date_20_days_ago.strftime("%Y-%m-%d")].reset_index(drop=True)

# The limited data could have a different number of queues
# calc_fig_height remains as calculated before
numqs = len(capqmon_publ_20.sendq.unique())

if numqs == 0:
    # Data frame could be empty
    print('No data available to display with MONITOR_TIME > ' + date_20_days_ago.strftime("%Y-%m-%d"))
else:
    capqmon_plt2 = plot_capqmon(capqmon_publ_20,numqs,deltadays)
    capqmon_plt2.show()        

Recent 5 days from IBMQREP_CAPQMON

In [ ]:
# Zoom in - Same figure as above, with data of the last 5 days only (maxmondate - 5 days)

# display 5 days of data only
deltadays = 5

# old: date_5_days_ago = date.today() - timedelta(days=deltadays)
date_5_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
   print('DEBUG: date_20_days_ago.strftime("%m/%d/%Y"): ' + date_20_days_ago.strftime("%Y-%m-%d"))

print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                       + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                       + maxmondate.strftime("%Y-%m-%d"))    

# New data frame - limited by the calculated data range 
#   - reset_index was introduced to guarantee xticks in the plot
capqmon_publ_5 = capqmon_publ[capqmon_publ.monitor_date 
                              >= date_5_days_ago.strftime("%Y-%m-%d")].reset_index(drop=True)

# The limited data could have a different number of queues
# calc_fig_height remains as calculated before
numqs = len(capqmon_publ_5.sendq.unique())

if numqs == 0:
    # Data frame could be empty
    print('No data available to display with MONITOR_TIME > ' + date_5_days_ago.strftime("%Y-%m-%d"))
else:    
    capqmon_plt3 = plot_capqmon(capqmon_publ_5,numqs,deltadays)
    capqmon_plt3.show()    

[Back to Top](#top)
<a id='qcap_details'></a>

# Q Capture details

## Send Queue details

Send queue details (IBMQREP_SENDQUEUES)

In [ ]:
# The function get_sendq_details executes a query which selects detail data from IBMQREP_SENDQUEUES
# and returns a pandas data frame
df_queues = get_sendq_details(connection, capture_schema)

# set index does not work
df_queues.set_index(['sendq'],inplace=True)
df_queues.sort_values(['sendq'], ascending=[True], inplace=True)

# printing the data frame
df_queues

[Back to Top](#top)
<a id='qsubs_details'></a>

## Subscription details

Subscription details (IBMQREP_SUBS). Use pixiedust options to filter the result set or to convert the table into a  graph.

In [ ]:
# The function get_subs_details executes a query which selects detail data from IBMQREP_SUBS
# and returns a pandas data frame
df_subs = get_subs_details(connection, capture_schema)

#df_subs.set_index('subname',inplace=True)
df_subs.sort_values(['subname'], ascending=[True], inplace=True)    

# displaying the data frame using pixiedust
display(df_subs)

[Back to Top](#top)
<a id='qsubs_qa'></a>

## Subscription quality assurance

Runtime error prevention for existing subscriptions. The following potential error situations are reported:
 <ul>
  <li>C-TNF - Source table not found</li>
  <li>C-CNF - Subscribed column does not exist in DB2</li>
  <li>C-CNS - Existing source column not subscribed</li>
  <li>C-DCC - Data capture flag missing for source table</li>    
</ul> 

In [ ]:
# The function get_capture_anomylies executes the query 'qrep_check_subs_capture.sql' 
# (with language dependent result set) and returns a pandas data frame
df_capqa = get_capture_anomylies(connection)

df_capqa.drop(df_capqa.columns[2], axis=1, inplace=True)

# conditional formatting for column 'sev'
df_capqa = (df_capqa.style
    .applymap(sev_background, subset=['sev'])
    .applymap(sev_foreground, subset=['sev'])
)

# printing the data frame
df_capqa

[Back to Top](#top)
<a id='qcap_cntl'></a>

# Q Capture Control Tables

## Q Capture parameters

Stored Q Capture parameters (IBMQREP_CAPPARMS)

In [ ]:
# The function get_capparms executes a query which selects detail data from IBMQREP_CAPPARMS
# and returns a pandas data frame
df_capparms = get_capparms(connection, capture_schema)

# printing the data frame
df_capparms

[Back to Top](#top)
<a id='qcap_mon'></a>

## Q Capture CAPMON and CAPQMON

In [ ]:
# Calculate the monitoring date range
# - as lowest date:
#      - in case custom_mon_start_date == -1 take current date - daterange days, otherweise custom_mon_start_date
# - as highest date:
#      - in case custom_mon_end_date == -1 take current date, otherweise custom_mon_start_date

calc_mon_dates = calc_mon_start_end(custom_mon_start_date, custom_mon_end_date, default_mon_date_range)

mon_start_date = calc_mon_dates[0]
mon_end_date   = calc_mon_dates[1]

# Debug
if debug:
    print("Monitor Start Date: " + mon_start_date)
    print("Monitor End Date:   " + mon_end_date)

In [ ]:
# The function get_capmon executes a query which selects detail data from IBMQREP_CAPMON
# and returns a pandas data frame
df_capmon = get_capmon(connection, capture_schema, mon_start_date, mon_end_date)

df_capmon.set_index('monitor_time')

# sort does not seem to work
df_capmon.sort_values(['monitor_time'], ascending=False, inplace=True)

# displaying the data frame using Pixiedust
display(df_capmon)

In [ ]:
# The function get_capqmon executes a query which selects detail data from IBMQREP_CAPQMON
# and returns a pandas data frame
df_capqmon = get_capqmon(connection, capture_schema, mon_start_date, mon_end_date)

# sort does not seem to work
df_capqmon.sort_values(['monitor_time'], ascending=False, inplace=True)

# displaying the data frame using Pixiedust
display(df_capqmon)

[Back to Top](#top)
<a id='qcap_trace'></a>

## Q Capture message log

Displaying the n (num_messages) most recent runtime log messages from IBMQREP_CAPTRACE 

In [ ]:
# The function get_captrace executes a query which selects detail data from IBMQREP_APPLYTRACE
# and returns a pandas data frame
df_captrace = get_captrace(connection, capture_schema, num_messages_trace)

# sort does not seem to work
df_captrace.sort_values(['trace_time'], ascending=False, inplace=True)

# displaying the data frame using Pixiedust
display(df_captrace)

[Back to Top](#top)
<a id='end'></a>

Done.

## Release Notes

<b>08.05.2019:</b> This is the initial release of the Jupyter Notebook for Q Capture monitoring. It is the beginning of a learning curve and uses various Python and Jupyter Notebook techniques such as:
 <ul>    
   <li>Db2 Python libraries</li>
   <li>Pandas data frames</li>
   <li>Pixiedust for result set visualization</li>
   <li>matplotlib for the graphical representation of performance statistics</li>
 </ul>    
<b>12.07.2019:</b> Fixed not displayed xticks for plots with limited date ranges (20 days, 5 days) by resetting the index after filtering the data frame (<i>.reset_index(drop=True)</i>)<p>
<b>02.12.2019:</b> Some queries (e.g., 'get_sendq_details') just retrieve the content of IBMQREP control tables. Previously, these SQL queries contained a list of columns (all columns). This was changed for multiple queries to first generate the list of columns from the Db2 catalog and then build and execute the query. This increases the robustness of the Notebook because with that feature it supports multiple control table architecture levels (prevents errors because of missing (optional) columns).<p>
<b>09.11.2020:</b> Improved DEBUG messages<br>
<b>09.11.2020:</b> Optimized tick density for performance figures

## Feedback

Feedback and ideas for improvement are welcome and can be sent to clenke@de.ibm.com